In [36]:
import pandas as pd
import numpy as np
import requests
import pickle
import sqlite3
import time
from tqdm import tqdm
import joblib
from sqlite3 import Error
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [50]:
def create_conn(db_file):
    
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
        
    except Error as e:
        print(e)
        
    return conn

conn = create_conn(r"C:\Users\bktor\Desktop\Python\DF\FANTASY.DB")
SQL = """
SELECT 
      Date, Starters, MP, FG, FGA, `FG%`, `3P`, `3PA`, FT, FTA, `FT%`, 
      ORB, DRB, TRB, AST, STL, BLK, TOV, PF, PTS, `+/-`, `TS%`, `eFG%`,
      `3PAr`, FTr, ORtg, DRtg, BPM, HCA, 
      PTS_DEF, TRB_DEF, TOV_DEF, ORTG_DEF
FROM MAIN_FINAL
WHERE MP not in ('Did Not Play' , 'Did Not Dress',
                'Player Suspended', 'Not With Team')
"""
df = pd.read_sql(SQL, con = conn)

def seconder(x):
    try:
        mins, secs = map(float, x.split(':'))
    except:
        secs = 0
        mins = 0
    return (mins * 60 + secs)

df['MP'] = df['MP'].apply(seconder)
columns = list(df.columns)
columns.pop(0)
columns.pop(0)
df[columns] = df[columns].apply(pd.to_numeric)

dd_list = []
for index, row in df.iterrows():
    stats = [row['PTS'], row['TRB'], row['BLK'],row['AST'],
             row['STL']]
    
    count = 0
    for stat in stats:
        if stat >= 10:
            count += 1
            
    if count == 2:
        dd_list.append(1.5)
    elif count > 2:
        dd_list.append(3)
    else:
        dd_list.append(0)

df['3P'] = .5 * df['3P']
df['ORB'] = 1.25 * df['ORB']
df['DRB'] = 1.25 * df['DRB']
df['TRB'] = 1.25 * df['TRB']
df['AST'] = 1.5 * df['AST']
df['STL'] = 2 * df['STL']
df['BLK'] = 2 * df['BLK']
df['TOV'] = -.5 * df['TOV']
df['DD_TD'] = dd_list

norm_columns = ['MP', 'PTS_DEF','TRB_DEF','TOV_DEF','ORTG_DEF','ORtg','DRtg']
for column in norm_columns:
    df[column] = (df[column] - df[column].min())/(df[column].max() - df[column].min())
df = df.fillna(0)
df['Score'] = df['3P'] + df['TRB'] + df['AST'] + df['STL'] + df['BLK'] + df['TOV'] + df['PTS'] + df['DD_TD']
df

2.6.0


,DATE,Starters,MP,FG,FGA,FG%,3P,3PA,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,TS%,eFG%,3PAr,FTr,ORtg,DRtg,BPM,HCA,PTS_DEF,TRB_DEF,TOV_DEF,ORTG_DEF,DD_TD,Score
0,2018-10-16 00:00:00,Al Horford,0.461006,4,7,0.571,0.0,1,1,1,1.0,0.00,5.00,5.00,3.0,0,8,-1.5,1,9,9.0,0.605,0.571,0.143,0.143,0.300000,0.854545,2.5,1,0.448690,0.414997,0.352227,0.443884,0.0,23.50
1,2018-04-01 00:00:00,Michael Kidd-Gilchrist,0.496665,6,13,0.462,0.0,0,4,4,1.0,1.25,5.00,6.25,0.0,4,0,-0.0,2,16,-21.0,0.542,0.462,0.000,0.308,0.386667,0.920000,0.0,1,0.453222,0.414944,0.342817,0.448310,0.0,26.25
2,2018-04-01 00:00:00,Kemba Walker,0.398409,2,7,0.286,0.5,5,5,5,1.0,0.00,1.25,1.25,6.0,0,2,-2.0,0,10,-12.0,0.543,0.357,0.714,0.714,0.296667,0.929091,-5.8,1,0.453222,0.414944,0.342817,0.448310,0.0,17.75
3,2018-10-16 00:00:00,Kyrie Irving,0.446383,2,14,0.143,0.5,8,2,2,1.0,0.00,5.00,5.00,10.5,0,0,-1.5,1,7,9.0,0.235,0.179,0.571,0.143,0.210000,0.870909,-9.0,1,0.448690,0.414997,0.352227,0.443884,0.0,21.50
4,2018-04-01 00:00:00,Dwight Howard,0.398153,4,7,0.571,0.0,0,2,5,0.4,1.25,11.25,12.50,0.0,2,2,-0.5,2,10,-13.0,0.543,0.571,0.000,0.714,0.336667,0.910909,-1.6,1,0.453222,0.414944,0.342817,0.448310,1.5,27.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107979,2016-12-31 00:00:00,Joe Ingles,0.350693,3,5,0.600,1.5,3,0,0,0.0,0.00,0.00,0.00,0.0,0,0,-1.5,0,9,2.0,0.900,0.900,0.600,0.000,0.313333,0.896364,-0.6,1,0.589423,0.469669,0.452303,0.508575,0.0,9.00
107980,2016-12-31 00:00:00,Trey Lyles,0.271421,1,5,0.200,0.0,3,1,2,0.5,0.00,5.00,5.00,3.0,0,0,-0.5,1,3,1.0,0.255,0.200,0.600,0.400,0.203333,0.885455,-7.2,1,0.589423,0.469669,0.452303,0.508575,0.0,10.50
107981,2016-12-31 00:00:00,Joe Johnson,0.266290,1,4,0.250,0.0,1,0,0,0.0,0.00,2.50,2.50,0.0,0,0,-1.0,3,2,5.0,0.250,0.250,0.250,0.000,0.110000,0.890909,-17.6,1,0.589423,0.469669,0.452303,0.508575,0.0,3.50
107982,2016-12-31 00:00:00,Raul Neto,0.216008,3,6,0.500,0.0,2,2,2,1.0,0.00,1.25,1.25,1.5,2,0,-0.0,2,8,7.0,0.581,0.500,0.333,0.333,0.413333,0.878182,5.5,1,0.589423,0.469669,0.452303,0.508575,0.0,12.75


In [51]:
from sklearn.preprocessing import MinMaxScaler

X_features = df.columns.tolist()
vals_skip = ['Score', 'Starters','DATE']
X_features = [value for value in X_features if value not in vals_skip]

X_df = df[X_features]
scaler_X = MinMaxScaler()
scaler_X.fit(X_df[X_features])
X_df[X_features] = scaler_X.transform(X_df[X_features])

Y_features = ['Score']
Y_df = df[Y_features]

scaler_Y = MinMaxScaler()
scaler_Y.fit(Y_df[Y_features])
Y_df[Y_features] = scaler_Y.transform(Y_df[Y_features])

df_scaled = pd.merge(X_df, Y_df, left_index=True, right_index=True)
df_scaled = df_scaled[[ col for col in df_scaled.columns if col != 'Score' ] + ['Score'] ]
df_scaled['Starters'] = df['Starters']
df_scaled['DATE'] = df['DATE']
df_scaled = df_scaled[ ['Starters'] + [ col for col in df.columns if col != 'Starters' ] ]
df_scaled

<ipython-input-51-d927fe09bcfe>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df[X_features] = scaler_X.transform(X_df[X_features])
c:\users\bktor\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
<ipython-input-51-d927fe09bcfe>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

,Starters,DATE,MP,FG,FGA,FG%,3P,3PA,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,TS%,eFG%,3PAr,FTr,ORtg,DRtg,BPM,HCA,PTS_DEF,TRB_DEF,TOV_DEF,ORTG_DEF,DD_TD,Score
0,Al Horford,2018-10-16 00:00:00,0.461006,0.173913,0.159091,0.571,0.000000,0.041667,0.038462,0.034483,1.0,0.000000,0.173913,0.133333,0.08,0.0,0.4,0.750000,0.166667,0.128571,0.542857,0.403333,0.380667,0.143,0.017875,0.300000,0.854545,0.660670,1.0,0.448690,0.414997,0.352227,0.443884,0.0,0.241546
1,Michael Kidd-Gilchrist,2018-04-01 00:00:00,0.496665,0.260870,0.295455,0.462,0.000000,0.000000,0.153846,0.137931,1.0,0.076923,0.173913,0.166667,0.00,0.2,0.0,1.000000,0.333333,0.228571,0.257143,0.361333,0.308000,0.000,0.038500,0.386667,0.920000,0.659022,1.0,0.453222,0.414944,0.342817,0.448310,0.0,0.268116
2,Kemba Walker,2018-04-01 00:00:00,0.398409,0.086957,0.159091,0.286,0.071429,0.208333,0.192308,0.172414,1.0,0.000000,0.043478,0.033333,0.16,0.0,0.1,0.666667,0.000000,0.142857,0.342857,0.362000,0.238000,0.714,0.089250,0.296667,0.929091,0.655200,1.0,0.453222,0.414944,0.342817,0.448310,0.0,0.185990
3,Kyrie Irving,2018-10-16 00:00:00,0.446383,0.086957,0.318182,0.143,0.071429,0.333333,0.076923,0.068966,1.0,0.000000,0.173913,0.133333,0.28,0.0,0.0,0.750000,0.166667,0.100000,0.542857,0.156667,0.119333,0.571,0.017875,0.210000,0.870909,0.653091,1.0,0.448690,0.414997,0.352227,0.443884,0.0,0.222222
4,Dwight Howard,2018-04-01 00:00:00,0.398153,0.173913,0.159091,0.571,0.000000,0.000000,0.076923,0.172414,0.4,0.076923,0.391304,0.333333,0.00,0.1,0.1,0.916667,0.333333,0.142857,0.333333,0.362000,0.380667,0.000,0.089250,0.336667,0.910909,0.657968,1.0,0.453222,0.414944,0.342817,0.448310,0.5,0.280193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107979,Joe Ingles,2016-12-31 00:00:00,0.350693,0.130435,0.113636,0.600,0.214286,0.125000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.750000,0.000000,0.128571,0.476190,0.600000,0.600000,0.600,0.000000,0.313333,0.896364,0.658627,1.0,0.589423,0.469669,0.452303,0.508575,0.0,0.101449
107980,Trey Lyles,2016-12-31 00:00:00,0.271421,0.043478,0.113636,0.200,0.000000,0.125000,0.038462,0.068966,0.5,0.000000,0.173913,0.133333,0.08,0.0,0.0,0.916667,0.166667,0.042857,0.466667,0.170000,0.133333,0.600,0.050000,0.203333,0.885455,0.654277,1.0,0.589423,0.469669,0.452303,0.508575,0.0,0.115942
107981,Joe Johnson,2016-12-31 00:00:00,0.266290,0.043478,0.090909,0.250,0.000000,0.041667,0.000000,0.000000,0.0,0.000000,0.086957,0.066667,0.00,0.0,0.0,0.833333,0.500000,0.028571,0.504762,0.166667,0.166667,0.250,0.000000,0.110000,0.890909,0.647423,1.0,0.589423,0.469669,0.452303,0.508575,0.0,0.048309
107982,Raul Neto,2016-12-31 00:00:00,0.216008,0.130435,0.136364,0.500,0.000000,0.083333,0.076923,0.068966,1.0,0.000000,0.043478,0.033333,0.04,0.1,0.0,1.000000,0.333333,0.114286,0.523810,0.387333,0.333333,0.333,0.041625,0.413333,0.878182,0.662647,1.0,0.589423,0.469669,0.452303,0.508575,0.0,0.137681


In [57]:
joblib.dump(scaler_X, 'scaler_x.pkl') 
joblib.dump(scaler_Y, 'scaler_y.pkl') 
df_scaled.to_csv("Test.csv")

df_op = pd.read_sql("SELECT * FROM MAIN_DEF", con = conn)
df_op.to_csv("DEF.csv")

In [58]:
df_op.sort_values('DATE_DEF')

,level_0,index,DATE_DEF,PTS_DEF,TRB_DEF,TOV_DEF,ORTG_DEF,TEAM
4473,4473,4473,2016-10-25 00:00:00,NaN,NaN,NaN,NaN,CLE
6788,6788,6788,2016-10-25 00:00:00,NaN,NaN,NaN,NaN,SAS
5459,5459,5459,2016-10-25 00:00:00,NaN,NaN,NaN,NaN,NYK
7776,7776,7776,2016-10-25 00:00:00,NaN,NaN,NaN,NaN,POR
2111,2111,2111,2016-10-25 00:00:00,NaN,NaN,NaN,NaN,GSW
...,...,...,...,...,...,...,...,...
10152,10152,0,2020-12-23 00:00:00,NaN,NaN,NaN,NaN,BOS
10157,10157,5,2020-12-23 00:00:00,NaN,NaN,NaN,NaN,TOR
10156,10156,4,2020-12-23 00:00:00,NaN,NaN,NaN,NaN,WAS
10166,10166,14,2020-12-23 00:00:00,NaN,NaN,NaN,NaN,ATL
